# Evaluation code for competition

## Load libraries

In [115]:
import os
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error

## Set variables

In [116]:
confirmed='confirmed'
deaths='deaths'
recovered='recovered'

data_dir='Track1_Submitted/'

## Load in data files

### Submitter (predicted)

In [117]:
unis = [x for x in os.listdir(data_dir) if '.DS' not in x]
submitter_dict = {}
for uni in unis:
    uni_dict = {}
    files = [x for x in os.listdir(data_dir+uni) if '.DS' not in x]
    cfile = [x for x in files if confirmed in x]
    dfile = [x for x in files if deaths in x]
    rfile = [x for x in files if recovered in x]
    if len(cfile)>0:
        uni_dict[confirmed] = pd.read_csv(data_dir+uni+'/'+cfile[0])
    if len(dfile)>0:
        uni_dict[deaths] = pd.read_csv(data_dir+uni+'/'+dfile[0])
    if len(rfile)>0:
        uni_dict[recovered] = pd.read_csv(data_dir+uni+'/'+rfile[0])
    submitter_dict[uni] = uni_dict

### JHU (true)

In [119]:
true_dte='5/1/20'
jhu_dict = {}
jhu_dict[confirmed] = \
(pd.
 read_csv('data/time_series_covid19_confirmed_global.csv').
 rename(columns = {true_dte : 'true'}).
 loc[:,['Province/State','Country/Region','true']].
 #only view countries/states with true counts>100
 query('true>100')
)
display(jhu_dict[confirmed].shape)
jhu_dict[deaths] = \
(pd.
 read_csv('data/time_series_covid19_deaths_global.csv').
 rename(columns = {true_dte : 'true'}).
 loc[:,['Province/State','Country/Region','true']].
 #only view countries/states with true counts>100
 query('true>100')
)
display(jhu_dict[deaths].shape)
jhu_dict[recovered] = \
(pd.
 read_csv('data/time_series_covid19_recovered_global.csv').
 rename(columns = {true_dte : 'true'}).
 loc[:,['Province/State','Country/Region','true']].
 #only view countries/states with true counts>100
 query('true>100')
)
display(jhu_dict[recovered].shape)

(191, 3)

(55, 3)

(154, 3)

## Comparing data files

In [120]:
pred_dte = '5/1/20'
results_dict = {}
types = [confirmed,recovered,deaths]
compare_dfs = {}
for uni in submitter_dict.keys():
    if uni!='sl4646':
        print(uni)
        uni_dict = {}
        for type_ in types:
            df_s = (submitter_dict[uni][type_].
                    set_index(['Province/State','Country/Region']))
            df_j = jhu_dict[type_].set_index(['Province/State','Country/Region'])
            compare_df = \
            (df_j[['true']].
             join(df_s[[pred_dte]].
                   rename(columns = {pred_dte : 'pred'}).
                   assign(pred = lambda x: x.pred.where(x.pred.ge(0))),
                 how='left')
            )
            compare_dfs[uni] = compare_df
            y_pred = (compare_df['pred'].
                      replace([-0,0],1).
                      replace([np.inf, -np.inf], np.nan).
                      replace([np.nan, -np.nan],1).
                      apply(np.log).
                      values)
            y_true = (compare_df['true'].
                      apply(np.log).
                      values)
            mse = mean_squared_error(y_true,y_pred) ** 0.5
            uni_dict[type_] = mse
        results_dict[uni] = uni_dict
results_df = pd.DataFrame(results_dict).T

manasi.sharma
dlm2202
mcg2208
a.saakyan
by2287


## Display RMSE ranking

In [121]:
display(results_df.sort_values(confirmed))

,confirmed,deaths,recovered
dlm2202,0.192450,0.132547,0.274031
mcg2208,0.214032,0.159250,0.400228
manasi.sharma,0.280215,0.158248,3.220875
by2287,0.710479,0.545783,1.475169
a.saakyan,0.850618,0.932349,1.384644


In [122]:
display(results_df.sort_values(deaths))

,confirmed,deaths,recovered
dlm2202,0.192450,0.132547,0.274031
manasi.sharma,0.280215,0.158248,3.220875
mcg2208,0.214032,0.159250,0.400228
by2287,0.710479,0.545783,1.475169
a.saakyan,0.850618,0.932349,1.384644


In [123]:
display(results_df.sort_values(recovered))

,confirmed,deaths,recovered
dlm2202,0.192450,0.132547,0.274031
mcg2208,0.214032,0.159250,0.400228
a.saakyan,0.850618,0.932349,1.384644
by2287,0.710479,0.545783,1.475169
manasi.sharma,0.280215,0.158248,3.220875


In [124]:
compare_dfs['dlm2202']

true   pred
Province/State   Country/Region                    
NaN              Algeria                 453    450
                 Argentina               225    237
                 Austria                 589    648
                 Bangladesh              170    223
                 Belgium                7703   8144
                 Brazil                 6412   5422
British Columbia Canada                  112    117
Ontario          Canada                 1265   1345
Quebec           Canada                 2022   2090
NaN              Chile                   234    259
Hubei            China                  4512   5049
NaN              Colombia                314    305
                 Czechia                 240    266
                 Denmark                 460    451
                 Dominican Republic      313    342
                 Ecuador                1063    789
                 Egypt                   406    383
                 Finland                 218    305
                 France                24563  24687
                 Germany                6736   7780
                 Greece                  140    152
                 Hungary                 323    337
                 India                  1223   1197
                 Indonesia               800    817
                 Iran                   6091   6345
                 Ireland                1265   1132
                 Israel                  225    261
                 Italy                 28236  28201
                 Japan                   455    573
                 Korea, South            250    259
                 Malaysia                103    106
                 Mexico                 1972   2065
                 Moldova                 122    108
                 Morocco                 171    220
                 Netherlands            4893   5417
                 Norway                  210    263
                 Pakistan                417    425
                 Panama                  192    205
                 Peru                   1124    991
                 Philippines             579    546
                 Poland                  651    642
                 Portugal               1007   1214
                 Romania                 744    771
                 Russia                 1169   1159
                 Saudi Arabia            169    229
                 Serbia                  179    160
                 South Africa            116    124
                 Spain                 24543  26046
                 Sweden                 2653   2959
                 Switzerland            1754   1777
                 Turkey                 3258   3508
                 Ukraine                 272    330
                 United Arab Emirates    111     83
                 United Kingdom        27510  23964
                 US                    64943  66184